In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
import datetime
if __name__ == "__main__":
    t1=datetime.datetime.now()
    appname = "FPgrowth"
    master ="local[4]" 
    
    data_list=[[['r', 'z', 'h', 'k', 'p']]\
               ,[['z', 'y', 'x', 'w', 'v', 'u', 't', 's']]\
               ,[['s', 'x', 'o', 'n', 'r']]\
               ,[['x', 'z', 'y', 'm', 't', 's', 'q', 'e']]\
               ,[['z']]\
               ,[['x', 'z', 'y', 'r', 'q', 't', 'p']]]#数据集
    conf = SparkConf().setAppName(appname).setMaster(master)  #spark配置                
    spark=SparkSession.builder.config(conf=conf).getOrCreate()
    data=spark.createDataFrame(data_list,["items"])#将数据转为spark中的dataframe
    fp = FPGrowth(minSupport=0.5, minConfidence=0.8)#模型建立
    fpm  = fp.fit(data)#模型拟合
    fpm .freqItemsets.show(5)#在控制台显示前五条频繁项集
    assRule=fpm.associationRules#强关联规则
    assRuleDf=assRule.toPandas()#转为python中的dataframe  
    print('强关联规则：\n',assRuleDf)
    new_data = spark.createDataFrame([(["s", "t"], )], ["items"])#新的前项数据
    print('后项预测：\n',fpm.transform(new_data).first().prediction) #预测后项               
    spark.stop()#关闭spark
    t2=datetime.datetime.now()
    print('spent ts:',t2-t1)

+---------+----+
|    items|freq|
+---------+----+
|      [t]|   3|
|   [t, x]|   3|
|[t, x, z]|   3|
|   [t, z]|   3|
|      [s]|   3|
+---------+----+
only showing top 5 rows

强关联规则：
    antecedent consequent  confidence  lift
0      [y, t]        [x]         1.0   1.5
1      [y, t]        [z]         1.0   1.2
2      [y, x]        [t]         1.0   2.0
3      [y, x]        [z]         1.0   1.2
4   [y, x, z]        [t]         1.0   2.0
5   [y, t, z]        [x]         1.0   1.5
6         [s]        [x]         1.0   1.5
7         [y]        [t]         1.0   2.0
8         [y]        [x]         1.0   1.5
9         [y]        [z]         1.0   1.2
10  [t, x, z]        [y]         1.0   2.0
11     [x, z]        [t]         1.0   2.0
12     [x, z]        [y]         1.0   2.0
13     [t, z]        [x]         1.0   1.5
14     [t, z]        [y]         1.0   2.0
15        [t]        [x]         1.0   1.5
16        [t]        [z]         1.0   1.2
17        [t]        [y]         1.0   2